# Market

In [47]:
symbols = 'SPY'
# symbols = ['SPY', 'GLD', 'SLV']

In [53]:
from quantopian.pipeline import Pipeline

def make_pipeline():
    return Pipeline()

pipeline = make_pipeline()

pipeline

In [70]:
from quantopian.research import run_pipeline

start_date = '2017-05-08'
end_date = start_date

universe = run_pipeline(pipeline, start_date, end_date)

universe.head()

Empty DataFrame
Columns: []
Index: [(2017-05-08 00:00:00+00:00, Equity(2 [ARNC])), (2017-05-08 00:00:00+00:00, Equity(21 [AAME])), (2017-05-08 00:00:00+00:00, Equity(24 [AAPL])), (2017-05-08 00:00:00+00:00, Equity(25 [ARNC_PR])), (2017-05-08 00:00:00+00:00, Equity(31 [ABAX]))]

# Position size

In [2]:
from datetime import datetime, timedelta

from pandas.tseries.offsets import BDay
from quantopian.research import get_pricing

PRICE_SLIDING_WINDOW = 57

end_date = datetime.now()
start_date = end_date - BDay(PRICE_SLIDING_WINDOW)

prices = get_pricing(
    symbols,
    start_date=start_date,
    end_date=end_date,
    fields=[
        'price',
        'high',
        'low',
        'close_price'
    ]
)

prices = prices.dropna()

prices.head()

,price,high,low,close_price
2017-02-16 00:00:00+00:00,233.723,234.136,232.837,233.723
2017-02-17 00:00:00+00:00,234.042,234.052,232.917,234.042
2017-02-21 00:00:00+00:00,235.426,235.665,234.490,235.426
2017-02-22 00:00:00+00:00,235.267,235.515,234.809,235.267
2017-02-23 00:00:00+00:00,235.416,235.874,234.540,235.416


In [3]:
from talib import ATR

AVERAGE_TRUE_RANGE_SLIDING_WINDOW = 21
MOVING_AVERAGE = 20

volatility = ATR(
    prices.high[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.low[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.close_price[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    timeperiod=MOVING_AVERAGE
)[-1]

volatility

1.4894999999999954

In [4]:
DOLLARS_PER_POINT = 1

dollar_volatility = volatility * DOLLARS_PER_POINT

dollar_volatility

1.4894999999999954

In [5]:
CAPITAL_PERCENTAGE_PER_TRADE = 0.01
START_CAPITAL = 1000000
CURRENT_CAPITAL = 900000
CAPITAL_MULTIPLIER = 2

profit = CURRENT_CAPITAL - START_CAPITAL
available_capital = START_CAPITAL

if profit < 0:
    available_capital = START_CAPITAL + profit * CAPITAL_MULTIPLIER

capital_per_trade = available_capital * CAPITAL_PERCENTAGE_PER_TRADE
trade_size = capital_per_trade / dollar_volatility

int(trade_size)

5370

# Entry

In [6]:
TWENTY_DAY_BREAKOUT = 20
FIFTY_FIVE_DAY_BREAKOUT = 55

In [7]:
twenty_day_high = prices.high[-TWENTY_DAY_BREAKOUT:].max()

twenty_day_high

239.72

In [8]:
fifty_five_day_high = prices.high[-FIFTY_FIVE_DAY_BREAKOUT:].max()

fifty_five_day_high

239.72

In [9]:
twenty_day_low = prices.low[-TWENTY_DAY_BREAKOUT:].min()

twenty_day_low

232.55000000000001

In [10]:
fifty_five_day_low = prices.low[-FIFTY_FIVE_DAY_BREAKOUT:].min()

fifty_five_day_low

231.61000000000001

In [11]:
latest_price = prices.price[-1]

latest_price

239.72

In [12]:
def get_system_one_entry_signal():
    """
    Get 20 day breakout entry signal.
    
    Does not factor in outcome of previous entry signal.
    """
    if latest_price > twenty_day_high:
        print('System one: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < twenty_day_low:
        print('System one: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System one: No signal')
        
get_system_one_entry_signal()

System one: No signal


In [13]:
def get_system_two_entry_signal():
    """
    Get 55 day breakout entry signal.
    """
    if latest_price > fifty_five_day_high:
        print('System two: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < fifty_five_day_low:
        print('System two: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System two: No signal')
        
get_system_two_entry_signal()

System two: No signal


In [14]:
latest_price = 240

get_system_one_entry_signal()

System one: Long SPY 5370@240.00


In [15]:
latest_price = 232

get_system_one_entry_signal()

System one: Short SPY 5370@232.00


In [16]:
latest_price = 240

get_system_two_entry_signal()

System two: Long SPY 5370@240.00


In [17]:
latest_price = 231

get_system_two_entry_signal()

System two: Short SPY 5370@231.00


In [18]:
def add_unit():
    """
    Add to position if price exceeds threshold.
    """
    previous_fill_price = 240

    long_threshold = previous_fill_price + volatility / 2
    short_threshold = previous_fill_price - volatility / 2

    if latest_price > long_threshold:
        print('Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < short_threshold:
        print('Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('No signal')

In [19]:
latest_price = 241

add_unit()

Long SPY 5370@241.00


In [20]:
latest_price = 239

add_unit()

Short SPY 5370@239.00


In [21]:
latest_price = 239.5

add_unit()

No signal


# Stop

In [22]:
STOP_MULTIPLIER = 2

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

237.02100000000002

In [23]:
# Re-enter if price hits entry price after stopped out
STOP_MULTIPLIER = 0.5

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

239.25524999999999

# Exit

In [24]:
TEN_DAY_EXIT_BREAKOUT = 10
TWENTY_DAY_EXIT_BREAKOUT = 20

In [25]:
ten_day_high = prices.high[-TEN_DAY_EXIT_BREAKOUT:].max()

ten_day_high

239.72

In [26]:
ten_day_low = prices.low[-TEN_DAY_EXIT_BREAKOUT:].min()

ten_day_low

236.61000000000001

In [27]:
twenty_day_high = prices.high[-TWENTY_DAY_EXIT_BREAKOUT:].max()

twenty_day_high

239.72

In [28]:
twenty_day_low = prices.low[-TWENTY_DAY_EXIT_BREAKOUT:].min()

twenty_day_low

232.55000000000001

In [35]:
def get_system_one_exit_signal():
    if position_type == 'long' and latest_price < ten_day_low \
            or position_type == 'short' and latest_price > ten_day_high:
        print('System one: Close %s' % symbols)
    else:
        print('System one: No signal')

In [36]:
def get_system_two_exit_signal():
    if position_type == 'long' and latest_price < twenty_day_low \
            or position_type == 'short' and latest_price > twenty_day_high:
        print('System two: Close %s' % symbols)
    else:
        print('System two: No signal')

In [37]:
position_type = 'long'
latest_price = 236

get_system_one_exit_signal()

System one: Close SPY


In [39]:
position_type = 'short'
latest_price = 240

get_system_one_exit_signal()

System one: Close SPY


In [42]:
position_type = 'long'
latest_price = 237

get_system_one_exit_signal()

System one: No signal


In [40]:
position_type = 'long'
latest_price = 232

get_system_two_exit_signal()

System two: Close SPY


In [43]:
position_type = 'short'
latest_price = 240

get_system_two_exit_signal()

System two: Close SPY


In [45]:
position_type = 'long'
latest_price = 233

get_system_two_exit_signal()

System two: No signal
